In [1]:
import torch
import torch
from torchvision.models import resnet18

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse

from loader import MoleculeDataset#################
#from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
import numpy as np

from model import GNN, GNN_graphpred,GNN_graphpred_1
from sklearn.metrics import roc_auc_score

from splitters import scaffold_split
import pandas as pd
import os
import shutil
from tensorboardX import SummaryWriter

In [3]:
class InteractionModel(torch.nn.Module):
    def __init__(self, protein_model,molecular_model,mol_emb_dim,pro_emb_dim, aggr = "add"):
        super( InteractionModel, self ).__init__() 
        
        self.batch_converter=protein_model.alphabet.get_batch_converter()
        
        #self.molecular.node_representation=molecular.model.gnn()
        #self.molecular.model,self.molecular.node_representation,self.molecular.features = GNN_graphpred_1(args.num_layer, args.emb_dim, num_tasks, JK = args.JK, drop_ratio = args.dropout_ratio, graph_pooling = args.graph_pooling, gnn_type = args.gnn_type)
        self.protein_sequence_representations = []
        self.pred_linear = torch.nn.Linear(pro_emb_dim+mol_emb_dim, num_tasks)
        
        def forward(self, protein_data,*molecular_data):
            ############protein
            batch_labels, batch_strs, batch_tokens = self.batch_converter(protein_data)
            batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
            with torch.no_grad():
                results = protein_model(batch_tokens, repr_layers=[33], return_contacts=True)
        
            protein_token_representations = results["representations"][33]
            
            for i, tokens_len in enumerate(batch_lens):
                self.protein_sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
            
            
            ################molecular
            
            #set up optimizer
            #different learning rate for different part of GNN
            molecular_node_representation=molecular_model.gnn(molecular_data.x,molecular_data.edge_index,molecular_data.edge_attr)
            molecular_representation=molecular_model.pool(molecular.node_representation,molecular_data[3])
            #molecular_representation=self.molecular.features(*molecular_data)
            
            
            
            all_features=torch.concat([self.protein_sequence_representations,molecular_representation],axis=1)
            self.pred_linear(all_features)